In [ ]:
from numpy.ma.testutils import assert_array_almost_equal
%matplotlib widget

In [ ]:
import gantt
from importlib import reload

from matplotlib import pyplot as plt
plt.rcParams['font.family'] = 'Nimbus Roman'


In [ ]:
df = gantt.load_gantt_df()

In [ ]:
gantt = reload(gantt)
gantt.STYLE = gantt.load_style()

gantt.layout_tasks(df)
gantt.layout_wp(df)
gantt.layout_secondment(df)

assert df['x0'].notna().all()

gantt.shrink_boxes(df, df['type'] == 'T', length=0.05, thickness=0.2)
gantt.shrink_boxes(df, df['type'] == 'WP', length=-0.05, thickness=-0.25)
gantt.shrink_boxes(df, df['type'] == 'S', length=0.0, thickness=-0.5)


def plotit(df):

    ax = gantt.create_fig()

    for wp, sdf in df.groupby('WP'):
        gantt.plot_wp_detailed(ax, sdf)

    gantt.plot_deliverables(ax, df[df['type'] == 'D'])
    gantt.plot_milestones(ax, df, df[df['type'] == 'M'])
    gantt.plot_secondments(ax, df[df['type'] == 'S'])

    ax.relim()
    ax.autoscale_view(tight=True)

    return ax


ax = plotit(df)
ax.figure.savefig('gantt.png', dpi=300, bbox_inches='tight')
;

In [ ]:
import numpy as np

def describe_item(item):
    name = item['name'].strip()
    t = item['type']
    desc = item['desc'].strip()
    start = int(item['start'])
    duration = int(item['duration'])
    stop = start + duration - 1

    suffix = ''

    if t == 'S':
        return np.nan

    if t in ['D', 'M']:
        prefix = '▸ '

    elif t in ['WP']:
        prefix = '\n*'
        suffix = '*'

    else:
        prefix = ''

    date = f'M{start}' if start == stop else f'M{start:d}-{stop:d}'

    return f'{prefix}{name} {desc} ({date}){suffix}'

print('\n'.join(df.apply(describe_item, axis=1).dropna().values))

